# Machine Learning - 2😍😍

## Minakov Nikita / CSSE1707-DA3/ 2020
## Lab #3 SVM

### Welcome to my diamonds analysis

In [30]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Context
This classic dataset contains the prices and other attributes of almost 54,000 diamonds. It's a great dataset for beginners learning to work with data analysis and visualization.

Content
price price in US dollars (\$326--\$18,823)

carat weight of the diamond (0.2--5.01)

cut quality of the cut (Fair, Good, Very Good, Premium, Ideal)

color diamond colour, from J (worst) to D (best)

clarity a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))

x length in mm (0--10.74)

y width in mm (0--58.9)

z depth in mm (0--31.8)

depth total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)

table width of top of diamond relative to widest point (43--95)

#### Import the dataset

In [31]:
dataset = pd.read_csv('diamonds.csv', )

In [32]:
dataset.head()


,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [33]:
dataset.corr()

,Unnamed: 0,carat,depth,table,price,x,y,z
Unnamed: 0,1.000000,-0.377983,-0.034800,-0.100830,-0.306873,-0.405440,-0.395843,-0.399208
carat,-0.377983,1.000000,0.028224,0.181618,0.921591,0.975094,0.951722,0.953387
depth,-0.034800,0.028224,1.000000,-0.295779,-0.010647,-0.025289,-0.029341,0.094924
table,-0.100830,0.181618,-0.295779,1.000000,0.127134,0.195344,0.183760,0.150929
price,-0.306873,0.921591,-0.010647,0.127134,1.000000,0.884435,0.865421,0.861249
x,-0.405440,0.975094,-0.025289,0.195344,0.884435,1.000000,0.974701,0.970772
y,-0.395843,0.951722,-0.029341,0.183760,0.865421,0.974701,1.000000,0.952006
z,-0.399208,0.953387,0.094924,0.150929,0.861249,0.970772,0.952006,1.000000


___________________________________________

 ### I will use only Ideal and Very good cut's of diamnond. Unneeded will be dropped

In [34]:
dataset.cut.value_counts().sort_index()

Fair          1610
Good          4906
Ideal        21551
Premium      13791
Very Good    12082
Name: cut, dtype: int64

In [35]:
dataset = dataset[dataset.cut!='Fair'] 
dataset = dataset[dataset.cut!='Good'] 
dataset = dataset[dataset.cut!='Premium'] 

In [36]:
dataset.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
5,6,0.24,Very Good,J,VVS2,62.8,57.0,336,3.94,3.96,2.48
6,7,0.24,Very Good,I,VVS1,62.3,57.0,336,3.95,3.98,2.47
7,8,0.26,Very Good,H,SI1,61.9,55.0,337,4.07,4.11,2.53
9,10,0.23,Very Good,H,VS1,59.4,61.0,338,4.00,4.05,2.39


'Very Good': 0, 
'Ideal': 1

In [37]:
mapping = {'Very Good': 0, 'Ideal': 1}
dataset = dataset.replace({'cut': mapping})

In [38]:
dataset = dataset.select_dtypes(exclude=['object'])
 

In [39]:
dataset.head()

,Unnamed: 0,carat,cut,depth,table,price,x,y,z
0,1,0.23,1,61.5,55.0,326,3.95,3.98,2.43
5,6,0.24,0,62.8,57.0,336,3.94,3.96,2.48
6,7,0.24,0,62.3,57.0,336,3.95,3.98,2.47
7,8,0.26,0,61.9,55.0,337,4.07,4.11,2.53
9,10,0.23,0,59.4,61.0,338,4.00,4.05,2.39


In [40]:
dataset_samp = dataset.sample(1000)
dataset_samp.head()

,Unnamed: 0,carat,cut,depth,table,price,x,y,z
15203,15204,1.01,0,63.1,60.0,6108,6.36,6.31,4.00
27883,27884,0.30,0,62.8,56.0,655,4.26,4.28,2.68
22822,22823,1.55,0,62.4,58.0,10851,7.36,7.42,4.61
37575,37576,0.46,1,61.4,56.0,990,4.97,4.99,3.06
13503,13504,1.01,0,61.5,61.0,5542,6.37,6.47,3.95


__________________


#### Preparation of dataset

In [41]:
dataset_samp.head()

,Unnamed: 0,carat,cut,depth,table,price,x,y,z
15203,15204,1.01,0,63.1,60.0,6108,6.36,6.31,4.00
27883,27884,0.30,0,62.8,56.0,655,4.26,4.28,2.68
22822,22823,1.55,0,62.4,58.0,10851,7.36,7.42,4.61
37575,37576,0.46,1,61.4,56.0,990,4.97,4.99,3.06
13503,13504,1.01,0,61.5,61.0,5542,6.37,6.47,3.95


In [42]:
X = dataset_samp.loc[:,['carat','depth', 'table', 'price', 'x', 'y', 'z']].values

In [43]:
X

array([[ 1.01, 63.1 , 60.  , ...,  6.36,  6.31,  4.  ],
       [ 0.3 , 62.8 , 56.  , ...,  4.26,  4.28,  2.68],
       [ 1.55, 62.4 , 58.  , ...,  7.36,  7.42,  4.61],
       ...,
       [ 0.28, 61.1 , 60.  , ...,  4.17,  4.21,  2.56],
       [ 0.93, 62.8 , 56.  , ...,  6.14,  6.22,  3.88],
       [ 0.4 , 62.6 , 57.  , ...,  4.68,  4.75,  2.95]])

In [44]:
len(X)

1000

In [45]:
y = dataset_samp.iloc[:,2].values

In [46]:
y

array([0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,

In [47]:
len(y)

1000

________________________________________________________

In [48]:
X

array([[ 1.01, 63.1 , 60.  , ...,  6.36,  6.31,  4.  ],
       [ 0.3 , 62.8 , 56.  , ...,  4.26,  4.28,  2.68],
       [ 1.55, 62.4 , 58.  , ...,  7.36,  7.42,  4.61],
       ...,
       [ 0.28, 61.1 , 60.  , ...,  4.17,  4.21,  2.56],
       [ 0.93, 62.8 , 56.  , ...,  6.14,  6.22,  3.88],
       [ 0.4 , 62.6 , 57.  , ...,  4.68,  4.75,  2.95]])

In [49]:
from sklearn.preprocessing import LabelEncoder

labelEncoderY = LabelEncoder()
y = labelEncoderY.fit_transform(y)
y

array([0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,

________________________________________________________________________________________


#### Spliting the dataset

In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

_________________________________________________________________________________

 ## SVM algorithm 

In [51]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')


In [ ]:
svclassifier.fit(X_train, y_train)